In [84]:
import numpy as np
import os
import scipy.stats as st

from collections import Counter

In [88]:


def conf_int_bs(prefix):
    bs_mas=[]  
    for i in range(len(os.listdir(prefix))):      
        with open(prefix+'LLH.bs='+str(i)+'.txt','r') as f:
            l=f.readlines()        
        l=[l[j].replace('\n','').replace(',','').split(' ') for j in range(len(l))]
    
        l=[[float(l[j][j1]) for j1 in range(len(l[0]))] for j in range(len(l))]    
        
        l=np.array(l).transpose()        
    
        bs_mas.append(l[-1][np.argmax(l[0])])
    
    
    a=bs_mas[1:]   #zero is without bootstrap 



    with open(prefix+'LLH.bs='+str(0)+'.txt','r') as f:
        l=f.readlines()        
    l=[l[j].replace('\n','').replace(',','').split(' ') for j in range(len(l))]
    l=[[float(l[j][j1]) for j1 in range(len(l[0]))] for j in range(len(l))]  
    l=np.array(l).transpose() 



    mr=[-1,-1] #migration prms
    if '12' in prefix:
        mr[0]=l[-2][np.argmax(l[0])] #get with the max LLH
    if '21' in prefix:
        mr[1]=l[-2][np.argmax(l[0])]
    if 'two' in prefix:
        mr[0], mr[1]=round(l[-3][np.argmax(l[0])],8), round(l[-2][np.argmax(l[0])],8)     

    #print(l[-1][np.argmax(l[0])], st.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=st.sem(a)))
    #print(mr,l[-1][np.argmax(l[0])])

    

    
    
    return st.t.interval(0.975, len(a)-1, loc=np.mean(a), scale=st.sem(a)), mr,l[-1][np.argmax(l[0])], a


In [94]:

def write_conf_int(dir):

    with open('./'+dir+'/'+dir+'.times.txt', 'r') as f:
        l=f.readlines()        
    l=[l[j].replace('\n','').split('\t') for j in range(1,len(l))]
    l=[[int(l[j][j1]) for j1 in range(len(l[0]))] for j in range(len(l))] 

    dct={}
    for j in range(len(l)-1):
        dct[l[j][0]]=[l[j][1], l[j+1][1]-1]

    Prefix=['./'+dir+'/LLH.no.mig.2/']
    migs=['migrations No.mig\t']
    
    
#    Prefix=['./'+dir+'/LLH.no.mig/', './'+dir+'/LLH.12/', './'+dir+'/LLH.21/', './'+dir+'/LLH.two.sided/']   
#    migs=['migrations No.mig\t', 'migration 1 to 2\t', 'migration 2 to 1\t', 'migration Two-sided\t']

    
    with open('./'+dir+'/'+dir+'.conf.int.txt','w') as f:
        f.write('#Migration_Type\t#Confident_Int\t#Mig_coef_12\t#Mig_coef_21\t#Tome_INTERVAL\t#From_true_sfs\t#From_bootstrap\n')
        for p, p1 in zip(Prefix, migs):

            
            
            pts=conf_int_bs(p)
            dct2=dict(Counter(pts[-1]))
            lf, rt =  pts[0]
 #           print(dct[int(lf)][1]-dct[int(lf)][0], (lf-int(lf)), dct[int(lf)][0])

            lf2 = dct[int(lf)][0]+(dct[int(lf)][1]-dct[int(lf)][0])*(lf-int(lf))
            rt2 = dct[int(rt)][0]+(dct[int(rt)][1]-dct[int(rt)][0])*(rt-int(rt))


            f.write(dir+' '+p1+str((round(lf2,3), round(rt2,3)))+'\t'+
                    str(pts[1][0])+'\t'+str(pts[1][1])+'\t'+
                    str(dct[pts[2]][0])+'\t'+str(pts[-2])+'\t'+str(dct2)+'\n')





In [95]:
DIR='san_din'
write_conf_int(DIR)

DIR='san_sar'
write_conf_int(DIR)

DIR='han_fre'
write_conf_int(DIR)

DIR='din_sar'
write_conf_int(DIR)